In [4]:
#Aukley's function 2-dimensions

import math
import random
import time
import numpy as np

from functools import cmp_to_key

#Chromosomes
L_chromosome = 8
N_chains=2**L_chromosome
#Lower and upper limits of search space
a = -15
b = 15

#Dimensions
N_dimensions = 2

crossover_point=int(L_chromosome/2)

#Now for each dimension we need a chromosome 'string'
def random_chromosome():
    #Now instead just one dimension we must use two so add an extra for loop could generalize use more than one dimension
    chromosome=[[],[]]
    #Remember than we are using more than one dimension and for each dimension we create a chromosome chain
    for dimension in range(0, N_dimensions):
      for i in range(0,L_chromosome):
          if random.random() < 0.1:
              chromosome[dimension].append(0)
          else:
              chromosome[dimension].append(1)

    return chromosome

#Number of chromosomes
N_chromosomes = 100
#probability of mutation
prob_m=0.5

F0=[]
fitness_values=[]

for i in range(0,N_chromosomes):
    F0.append(random_chromosome())
    fitness_values.append(0)

#binary codification
def decode_chromosome(chromosome):
    global L_chromosome, N_chains, a, b
    #We have two dimensions again so, is mandatory to evaluate each chromosome chain
    value = [0.0, 0.0]
    
    for dimension in range(0, N_dimensions):
      chain_value = 0.0
      for p in range(L_chromosome):
          chain_value += (2**p) * chromosome[dimension][-1-p]
      value[dimension] = a + (b - a) * float(chain_value) / (N_chains - 1)

    return value


#As we are using two dimensions the function now uses two parameters x and y
def f(x, y):
    return -20 * math.exp(-0.2 * math.sqrt(0.5 * (x**2 + y**2))) - math.exp(0.5 * (math.cos(2 * math.pi * x) + math.cos(2 * math.pi * y))) + math.e + 20



def evaluate_chromosomes():
    global F0

    for p in range(N_chromosomes):
        value = decode_chromosome(F0[p])
        fitness_values[p] = f(value[0], value[1])
        

def compare_chromosomes(chromosome1,chromosome2):
    result_fx1 = decode_chromosome(chromosome1)
    result_fx2 = decode_chromosome(chromosome2)
    value1 = f(result_fx1[0], result_fx1[1])
    value2 = f(result_fx2[0], result_fx2[1])
    if value1 > value2:
        return 1
    elif value1 == value2:
        return 0
    else: #value1 < value2
        return -1


suma = float(N_chromosomes*(N_chromosomes+1))/2.

Lwheel = N_chromosomes*10

def create_wheel():
    global F0, fitness_values

    maxv = max(fitness_values)

    acc = 0
    for p in range(N_chromosomes):
        acc += maxv - fitness_values[p]

    fraction = []
    for p in range(N_chromosomes):
        fraction.append(float(maxv - fitness_values[p]) / acc)
        if fraction[-1] <= 1.0 / Lwheel:
            fraction[-1] = 1.0 / Lwheel

##    print fraction
    fraction[0] -= (sum(fraction)-1.0)/2
    fraction[1] -= (sum(fraction)-1.0)/2
##    print fraction

    wheel=[]

    pc = 0

    for f in fraction:
        Np=int(f*Lwheel)
        for i in range(Np):
            wheel.append(pc)
        pc+=1

    return wheel
        
F1=F0[:]
n=0
def nextgeneration():
    global n
    
    F0.sort(key=cmp_to_key(compare_chromosomes) )
    print( "Best solution so far:")
    n+=1
    result = decode_chromosome(F0[0])
    print(f'{n} f({result}) = {f(result[0], result[1])}')
                                                                    
    #elitism, the two best chromosomes go directly to the next generation
    F1[0]=F0[0]
    F1[1]=F0[1]
    roulette=create_wheel()
    for i in range(0,int((N_chromosomes-2)/2)):
        #Two parents are selected
        p1=random.choice(roulette)
        p2=random.choice(roulette)
        #Two descendants are generated but again we are using two dimensions
        o1 = [[], []]
        o2 = [[], []]
        for dimension in range(0, N_dimensions):
          o1[dimension] = F0[p1][dimension][0:crossover_point]
          o1[dimension].extend(F0[p2][dimension][crossover_point:L_chromosome])
          o2[dimension] = F0[p2][dimension][0:crossover_point]
          o2[dimension].extend(F0[p1][dimension][crossover_point:L_chromosome])
          #Each descendant is mutated with probability prob_m
          if random.random() < prob_m:
              o1[dimension][int(round(random.random()*(L_chromosome-1)))] ^= 1
          if random.random() < prob_m:
              o2[dimension][int(round(random.random()*(L_chromosome-1)))] ^= 1
        #The descendants are added to F1
        F1[2+2*i] = o1
        F1[3+2*i] = o2

    #The generation replaces the old one
    F0[:]=F1[:]


x = list(map(decode_chromosome,F0))
y_population = np.zeros(N_chromosomes)
F0.sort(key=cmp_to_key(compare_chromosomes))
evaluate_chromosomes()

for i in range(0, 10):
  nextgeneration()

Best solution so far:
1 f([5.5882352941176485, -0.5294117647058822]) = 13.27624521441775
Best solution so far:
2 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
3 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
4 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
5 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
6 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
7 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
8 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
9 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006
Best solution so far:
10 f([-0.0588235294117645, -0.0588235294117645]) = 0.4114144574357006


In [6]:
#Rastringin's function 3-dimensions

import math
import random
import time
import numpy as np

from functools import cmp_to_key

#Chromosomes
L_chromosome = 8
N_chains=2**L_chromosome
#Lower and upper limits of search space
a = -10
b = 10
#Dimensions
N_dimensions = 3

crossover_point=int(L_chromosome/2)

#Now for each dimension we need a chromosome 'string'
def random_chromosome():
    #Now instead just one dimension we must use three so add an extra for loop could generalize use more than one dimension
    chromosome=[[],[],[]]
    #Remember than we are using more than one dimension and for each dimension we create a chromosome chain
    for dimension in range(0, N_dimensions):
      for i in range(0, L_chromosome):
          if random.random() < 0.5:
              chromosome[dimension].append(0)
          else:
              chromosome[dimension].append(1)

    return chromosome

#Number of chromosomes
N_chromosomes = 300
#probability of mutation
prob_m = 0.5

F0=[]
fitness_values=[]

for i in range(0,N_chromosomes):
    F0.append(random_chromosome())
    fitness_values.append(0)

#binary codification
def decode_chromosome(chromosome):
    global L_chromosome,N_chains,a,b
    #We have three dimensions again so, is mandatory to evaluate each chromosome chain
    value = [0.0, 0.0, 0.0]
    
    for dimension in range(0, N_dimensions):
      chain_value = 0.0
      for p in range(L_chromosome):
          chain_value += (2**p) * chromosome[dimension][-1-p]
      value[dimension] = a + (b - a) * float(chain_value) / (N_chains - 1)

    return value


#As we are using two dimensions the function now uses two parameters x and y
def f(x, y, z):
    return 30 + x**2 + y**2 + z**2 - 10 * math.cos(2 * math.pi * x) - 10 * math.cos(2 * math.pi * y) - 10 * math.cos(2 * math.pi * z)

def evaluate_chromosomes():
    global F0

    for p in range(N_chromosomes):
        value = decode_chromosome(F0[p])
        fitness_values[p] = f(value[0], value[1], value[2])
        

def compare_chromosomes(chromosome1,chromosome2):
    result_fx1 = decode_chromosome(chromosome1)
    result_fx2 = decode_chromosome(chromosome2)

    value1 = f(result_fx1[0], result_fx1[1], result_fx1[2])
    value2 = f(result_fx2[0], result_fx2[1], result_fx2[2])
    if value1 > value2:
        return 1
    elif value1 == value2:
        return 0
    else: #value1 < value2
        return -1


suma = float(N_chromosomes*(N_chromosomes+1))/2.

Lwheel = N_chromosomes*10

def create_wheel():
    global F0,fitness_values

    maxv=max(fitness_values)
    acc=0
    for p in range(N_chromosomes):
        acc+=maxv-fitness_values[p]
    fraction=[]
    for p in range(N_chromosomes):
        fraction.append( float(maxv-fitness_values[p])/acc)
        if fraction[-1]<=1.0/Lwheel:
            fraction[-1]=1.0/Lwheel
##    print fraction
    fraction[0]-=(sum(fraction)-1.0)/2
    fraction[1]-=(sum(fraction)-1.0)/2
##    print fraction

    wheel=[]

    pc=0

    for f in fraction:
        Np=int(f*Lwheel)
        for i in range(Np):
            wheel.append(pc)
        pc+=1

    return wheel
        
F1=F0[:]
n=0
def nextgeneration():
    global n
    
    F0.sort(key=cmp_to_key(compare_chromosomes) )
    print( "Best solution so far:")
    n+=1
    result = decode_chromosome(F0[0])
    print(f'{n} f({result}) = {f(result[0], result[1], result[2])}')
                                                                    
    #elitism, the two best chromosomes go directly to the next generation
    F1[0]=F0[0]
    F1[1]=F0[1]
    roulette=create_wheel()
    for i in range(0,int((N_chromosomes-2)/2)):
        #Two parents are selected
        p1=random.choice(roulette)
        p2=random.choice(roulette)
        #Two descendants are generated but again we are using three dimensions
        o1 = [[], [], []]
        o2 = [[], [], []]
        for dimension in range(0, N_dimensions):
          o1[dimension] = F0[p1][dimension][0:crossover_point]
          o1[dimension].extend(F0[p2][dimension][crossover_point:L_chromosome])
          o2[dimension] = F0[p2][dimension][0:crossover_point]
          o2[dimension].extend(F0[p1][dimension][crossover_point:L_chromosome])
          #Each descendant is mutated with probability prob_m
          if random.random() < prob_m:
              o1[dimension][int(round(random.random()*(L_chromosome-1)))] ^= 1
          if random.random() < prob_m:
              o2[dimension][int(round(random.random()*(L_chromosome-1)))] ^= 1
        #The descendants are added to F1
        F1[2+2*i] = o1
        F1[3+2*i] = o2

    #The generation replaces the old one
    F0[:]=F1[:]


x = list(map(decode_chromosome,F0)) 
y_population = np.zeros(N_chromosomes)
F0.sort(key=cmp_to_key(compare_chromosomes))
evaluate_chromosomes()

for i in range(0, 100):
  nextgeneration()

Best solution so far:
1 f([3.882352941176471, -0.039215686274509665, -0.9803921568627452]) = 19.0231073818735
Best solution so far:
2 f([3.882352941176471, -0.039215686274509665, -0.9803921568627452]) = 19.0231073818735
Best solution so far:
3 f([-2.1568627450980395, 1.0588235294117645, -0.039215686274509665]) = 11.228360653679115
Best solution so far:
4 f([0.9019607843137258, -0.9803921568627452, 2.0]) = 7.688527817396235
Best solution so far:
5 f([0.9019607843137258, -0.9803921568627452, 2.0]) = 7.688527817396235
Best solution so far:
6 f([0.9019607843137258, -0.9803921568627452, 2.0]) = 7.688527817396235
Best solution so far:
7 f([0.9019607843137258, -0.9803921568627452, 2.0]) = 7.688527817396235
Best solution so far:
8 f([0.039215686274509665, -0.9019607843137258, 0.9803921568627452]) = 3.9920963270961156
Best solution so far:
9 f([0.039215686274509665, -0.9019607843137258, 0.9803921568627452]) = 3.9920963270961156
Best solution so far:
10 f([0.039215686274509665, -0.90196078431372